# PlantDisPre 
[link for working prototype: https://plantdispre.herokuapp.com](https://plantdispre.herokuapp.com)
## an application to predict the health  and disease of a plant
## providing natural remidies to prevent those problems

Cite: We will use the [PlantVillage dataset](https://github.com/spMohanty/PlantVillage-Dataset/tree/master/data_distribution_for_SVM) and it is a PyTorch implementation of [Using Deep Learning for Image-Based Plant Disease Detection](https://arxiv.org/ftp/arxiv/papers/1604/1604.03169.pdf)


We will be using Densenet121 and using transfer learning approach to train the neural network

installing PyTorch and loading the dataset from Google Drive

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
from google.colab import drive
drive.mount('/gdrive')
path = '/gdrive/My Drive/plant'


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


importing torch and checking for availability of cuda

In [0]:
# Imports here
import torch
from torch import nn
from torch import optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

import numpy as np
from torchvision import transforms, models, datasets

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


## Loading the Data

Here we are using `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html))
the data is divided into train and validation sets

For training to attain generalisation and to overcomme overfitting we are goig to apply some transforms  such as random rotation, random resizing and croping, flipping etcetera.

As validation set is used to measure the  performance of model on the data  it hasn't seen. So we only resize and crop to appropriate size.

As The pre-trained networks available from `torchvision` were trained on the ImageNet dataset where each color channel was normalized separately. 
So For both sets we  need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.

In [0]:
data_dir = path 
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'

# TODO: Define your transforms for the training and validation sets
#data_transforms = 
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224, scale=(0.7, 1.0), ratio=(0.75, 1.3333333333333333), interpolation=2),
                                       transforms.ColorJitter(.15,.15,.15,.15),
       
                                       transforms.RandomHorizontalFlip(),
                                       transforms.RandomVerticalFlip(),
                                       transforms.ToTensor(),
                                     
                                       transforms.Normalize([.485,.456,.406],
                                                            [.229,.224,.225])])
valid_transforms = transforms.Compose([transforms.Resize(240),
                                       transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize([.485,.456,.406],
                                                            [.229,.224,.225])])


# TODO: Load the datasets with ImageFolder
#image_datasets = 
train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
valid_data = datasets.ImageFolder(valid_dir, transform=valid_transforms)

# TODO: Using the image datasets and the trainforms, define the dataloaders
#dataloaders = 
batchSize = 32
trainloader = torch.utils.data.DataLoader(train_data, batch_size=batchSize, shuffle=True)
validloader = torch.utils.data.DataLoader(valid_data, batch_size=batchSize)


using Densenet121 and chaning the classifier part with two hidden layers. Here Number of outputs is 38.
We wull be using Adam as optimizer and ReduceLROnPlateau as scheduler

In [ ]:
# TODO: Build and train your network
model = models.densenet121(pretrained = True)
from collections import OrderedDict

for param in model.parameters():
    param.required_grad = False

classifier = nn.Sequential(OrderedDict([
                                ('fc1',nn.Linear(1024,512)),
                                ('relu',nn.ReLU()),
                                ('fc2',nn.Linear(512,38)),                            
                                ('output',nn.LogSoftmax(dim=1))]))
model.classifier = classifier
criterion = nn. NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(),lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3)

#checking for GPU
if train_on_gpu:
    model.cuda()

In [0]:
n_epochs = 15


valid_loss_min = np.Inf

for epoch in range(1,n_epochs+1):
    
    train_loss = 0.0
    valid_loss = 0.0
    accuracy = 0.0
    model.train()
    for data, target in trainloader:
        #if GPU is available move them to GPU
        if train_on_gpu:
            data,target  = data.cuda(), target.cuda()
        #clear the gradients of all optimized variables
        optimizer.zero_grad()
        
        #Computing the outputs by psiing inputs to the model 
        output = model(data)
        #calculating batch loss
        loss = criterion(output, target)
        #compute the gradients for backpropogation
        loss.backward()
        #updating the parameters by optimization step
        optimizer.step()
        #Update training loss
        train_loss+= loss.item()*data.size(0)
        
    #validation phase
    model.eval()
    for data, target in validloader:
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        output = model(data)
        loss = criterion(output, target)
        valid_loss+= loss.item()*data.size(0)
        #accuracy
        ps = torch.exp(output)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == target.view(*top_class.shape)
        accuracy += torch.sum(equals.type(torch.FloatTensor)).item()

    train_loss = train_loss/len(trainloader.dataset)
    valid_loss = valid_loss/len(validloader.dataset)
    accuracy = accuracy/len(validloader.dataset)
    scheduler.step(valid_loss)

    #printing training and validation statistics
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}\tValidation Accuracy: {:.6f}'.format(
        epoch, train_loss, valid_loss, accuracy))
    #saving the model if validation loss has decreased
    if valid_loss<=valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'plantDensenet121.pt')
        valid_loss_min = valid_loss
        

Epoch: 1 	Training Loss: 1.309208 	Validation Loss: 0.607066	Validation Accuracy: 0.806011
Validation loss decreased (inf --> 0.607066).  Saving model ...
Epoch: 2 	Training Loss: 0.533311 	Validation Loss: 0.388867	Validation Accuracy: 0.870959
Validation loss decreased (0.607066 --> 0.388867).  Saving model ...
Epoch: 3 	Training Loss: 0.415532 	Validation Loss: 0.321552	Validation Accuracy: 0.892197
Validation loss decreased (0.388867 --> 0.321552).  Saving model ...
Epoch: 4 	Training Loss: 0.367352 	Validation Loss: 0.314290	Validation Accuracy: 0.892576
Validation loss decreased (0.321552 --> 0.314290).  Saving model ...
Epoch: 5 	Training Loss: 0.357933 	Validation Loss: 0.278741	Validation Accuracy: 0.904049
Validation loss decreased (0.314290 --> 0.278741).  Saving model ...
Epoch: 6 	Training Loss: 0.316591 	Validation Loss: 0.245641	Validation Accuracy: 0.919503
Validation loss decreased (0.278741 --> 0.245641).  Saving model ...
Epoch: 7 	Training Loss: 0.309144 	Validation

# testing accuracy


In [0]:
correct = 0
total = 0
with torch.no_grad():
    for data, targets in testloader:
        if train_on_gpu:
            data, targets = data.cuda(), targets.cuda()
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total+=targets.size(0)
        correct += (predicted == targets).sum().item()
        
print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the test images: 87 %
